<a href="https://colab.research.google.com/github/ivkinivan/BlueDrapes/blob/main/Finver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# ПП 


In [ ]:
import numpy 
print("Выберите необходимый тип генерации 1)новости   2)сочинение   3)статья ")
generation_type = int(input())

Выберите необходимый тип генерации 1)новости   2)сочинение   3)статья 
1


In [ ]:
if (generetion_type == 1):
  print("")
  #do // отпралвяем на обученную модель 1 
elif (generetion_type == 2):
  print("")
  #do ///отпралвяем на обученную модель 2
elif (generetion_type == 3):
  print("")
  #do ///отпралвяем на обученную модель 3
#else print("Некорректно ввели тип")

NameError: ignored

In [ ]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
TRAIN_TEXT_FILE_PATH = "hm59v2.txt"

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'е', 'а', 'и', 'т', 'н', 'с', 'р', 'в', 'л', 'м', 'д', 'к', 'у', 'п', ',', 'ы', 'я', 'ь', 'б', 'ч', 'г', 'з', '.', 'ж', 'й', 'ю', 'х', 'ш', 'щ', 'ц', 'э', '–', 'Н', '«', '»', 'В', 'П', 'М', 'ф', 'С', 'К', 'О', ':', 'И', '-', '\n', 'А', 'Ч', 'Д', '?', 'Т', 'Р', 'Г', 'Л', 'Б', '!', '…', 'З', 'Я', 'Э', 'У', 'Е', 'Х', ';', 'ъ', '(', ')', 'Ж', 'Ф', 'Ш', 'I', '1', 'X', 'Ц', '0', '2', '9', '4', 'N', 'Ю', '7', '5', '8', '3', '“', '”', '6', 'C', 'F', 'A', 'V', '—']


In [ ]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [ ]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [ ]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 50000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)



Loss: 3.0641301250457764
 вобосто постом даде порода но на сто сто восто но рабом вобосто босто сто сромом сто потоволае сто вовосто восто на дасто побостисто досто вода насто посто достиза сто сакосто на но сто подем на повар
Loss: 2.4754958820343016
 жем нановать кодым на не ворения на кото помоде словенновень стовения обовет посторывает своем на стороветь на подела поления ворения не пода на на в и дастать вольдения сторых в соверется в котого по
Loss: 2.2489934539794922
 леса постали своем все помодит содониние содного собразный полодым и свое посторые странием помодит на доброми подда как подно все попомения стором посторова продного постание подной соблествой с помо
Loss: 2.1227581214904787
 подумать в своей своем станивать нас на нашей сама нашественные человек на собрать добрать в не метерать в нас не станивает поставить постать собольно в добрать нас как полнеть нас принительно обласии
Loss: 2.038863892555237
 старов и стальное не с верет в недал полова природы подобрать сомно

KeyboardInterrupt: ignored

In [ ]:
import pickle
torch.save(model.state_dict(), 'model.pt')

with open('char_to_idx.pickle', 'wb') as f: pickle.dump(char_to_idx, f)

with open('idx_to_char.pickle', 'wb') as f: pickle.dump(idx_to_char, f)

In [ ]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=40, 
    start_text='Как тебя починить? '
    )
)


Как тебя починить? не может быть все время не принимать от 
